# Introduction

This file illustrates the process of creation of DH homogenious transformation matrices

In [20]:
from sympy import init_session, symbols, sin, cos, Matrix, pi, trigsimp

# 2. Denavit-Hartenberg (DH) Parameters Derivation

### Create mathematical symbols

In [2]:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

### Create Generalized form of DH homogenious transform matrix
Generalize DH homogenious transform matrix is based on [Udacity Forward Kinematics Class](https://classroom.udacity.com/nanodegrees/nd209/parts/c199593e-1e9a-4830-8e29-2c86f70f489e/modules/undefined/lessons/87c52cd9-09ba-4414-bc30-24ae18277d24/concepts/c0837700-3de6-4c41-8a5d-1e25936e0cdb)
![DH homogenious transform matrix](supporting_images/generic_dh_tf.png)

In [3]:
def tf_matrix(alpha, a, d, q):
  # Creates a transformation matrix based yaw, pitch, roll rotations
  # for around x, y, z rotations
  tf = Matrix([
    [cos(q), -sin(q), 0, a],
    [sin(q) * cos(alpha), cos(q) * cos(alpha), -sin(alpha), -sin(alpha) * d],
    [sin(q) * sin(alpha), cos(q) * sin(alpha), cos(alpha), cos(alpha) * d],
    [0, 0, 0, 1]
  ])
  return tf

### Create table of DH parameters

In [4]:
dh_table = {
  alpha0:     0,  a0:        0, d1:  0.75, q1: q1,
  alpha1: -pi/2,  a1:     0.35, d2:     0, q2: q2-pi/2,
  alpha2:     0,  a2:     1.25, d3:     0, q3: q3,
  alpha3: -pi/2,  a3:   -0.054, d4:   1.5, q4: q4,
  alpha4:  pi/2,  a4:        0, d5:     0, q5: q5,
  alpha5: -pi/2,  a5:        0, d6:     0, q6: q6,
  alpha6:     0,  a6:        0, d7: 0.303, q7:   0
}

### Create transformation matrices for each joint and a homogeneous transform matrix from base_link to gripper_link

In [10]:
t01     = tf_matrix(alpha0, a0, d1, q1).subs(dh_table)
t01

Matrix([
[1, 0, 0,     0],
[0, 1, 0,     0],
[0, 0, 1, 0.303],
[0, 0, 0,     1]])

In [9]:
t12     = tf_matrix(alpha1, a1, d2, q2).subs(dh_table)
t12

Matrix([
[sin(q2),  cos(q2), 0, 0.35],
[      0,        0, 1,    0],
[cos(q2), -sin(q2), 0,    0],
[      0,        0, 0,    1]])

In [12]:
t23     = tf_matrix(alpha2, a2, d3, q3).subs(dh_table)
t23

Matrix([
[cos(q3), -sin(q3), 0, 1.25],
[sin(q3),  cos(q3), 0,    0],
[      0,        0, 1,    0],
[      0,        0, 0,    1]])

In [13]:
t34     = tf_matrix(alpha3, a3, d4, q4).subs(dh_table)
t34

Matrix([
[ cos(q4), -sin(q4), 0, -0.054],
[       0,        0, 1,    1.5],
[-sin(q4), -cos(q4), 0,      0],
[       0,        0, 0,      1]])

In [14]:
t45     = tf_matrix(alpha4, a4, d5, q5).subs(dh_table)
t45

Matrix([
[cos(q5), -sin(q5),  0, 0],
[      0,        0, -1, 0],
[sin(q5),  cos(q5),  0, 0],
[      0,        0,  0, 1]])

In [15]:
t56     = tf_matrix(alpha5, a5, d6, q6).subs(dh_table)
t56

Matrix([
[ cos(q6), -sin(q6), 0, 0],
[       0,        0, 1, 0],
[-sin(q6), -cos(q6), 0, 0],
[       0,        0, 0, 1]])

In [16]:
t6_ee   = tf_matrix(alpha6, a6, d7, q7).subs(dh_table)
t6_ee

Matrix([
[1, 0, 0,     0],
[0, 1, 0,     0],
[0, 0, 1, 0.303],
[0, 0, 0,     1]])

In [21]:
t0_ee   = t01 * t12 * t23 * t34 * t45 * t56 * t6_ee
t0_ee = trigsimp(t0_ee)
t0_ee

Matrix([
[((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) - (-sin(q1)*cos(q4) + sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6), -((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) - (-sin(q1)*cos(q4) + sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6), -(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), -0.303*sin(q1)*sin(q4)*sin(q5) + 1.25*sin(q2)*cos(q1) - 0.303*sin(q5)*sin(q2 + q3)*cos(q1)*cos(q4) - 0.054*sin(q2 + q3)*cos(q1) + 0.303*cos(q1)*cos(q5)*cos(q2 + q3) + 1.5*cos(q1)*cos(q2 + q3) + 0.35*cos(q1)],
[ ((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*sin(q6),  -((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*cos(q6), -(sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*si